In [1]:
from sklearn.preprocessing import LabelBinarizer

from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, model_from_json
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.constraints import maxnorm
from keras import regularizers
from keras.layers.normalization import BatchNormalization
from keras.layers import Dense, Dropout, Activation, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, SeparableConv2D
from keras.layers import concatenate, add, Lambda
from keras.models import Model
from keras.regularizers import l2

from keras.applications import imagenet_utils, Xception
from keras.preprocessing.image import img_to_array
import keras.backend as K

import numpy as np 
import json
import os
import cv2
import h5py

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
from helpers import TrainingMonitor
from helpers import Utils

In [3]:
output_path = "../output/"

In [4]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [5]:
db_train = h5py.File("../input/datasets/UCF-101-hdf5/rgbmean_train.hdf5")
db_val = h5py.File("../input/datasets/UCF-101-hdf5/rgbmean_val.hdf5")
db_test = h5py.File("../input/datasets/UCF-101-hdf5/rgbmean_test.hdf5")

x_train_rgbmean = db_train["images"][:]
x_val_rgbmean = db_val["images"][:]
x_test_rgbmean = db_test["images"][:]

y_train_rgbmean = db_train["labels"][:]
y_val_rgbmean = db_val["labels"][:]
y_test_rgbmean = db_test["labels"][:]

In [6]:
num_classes = y_test_rgbmean.shape[1]
num_classes

8

In [7]:
class XCEPTION:
    @staticmethod
    def separable_module(x, filters, reg=0.0005):
        residual = Conv2D(filters, (1, 1), strides=(2, 2), padding='same', use_bias=False, kernel_regularizer=l2(reg))(x)
        residual = BatchNormalization()(residual)
        
        x = SeparableConv2D(filters, (3, 3), padding='same', use_bias=False, kernel_regularizer=l2(reg))(x)
        x = Activation('relu')(x)
        x = BatchNormalization(axis=-1)(x)
        x = SeparableConv2D(filters, (3, 3), padding='same', use_bias=False, kernel_regularizer=l2(reg))(x)
        x = BatchNormalization(axis=-1)(x)
        x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
        x = add([x, residual])
        return x
        
    @staticmethod
    def build(include_top=True, reg=0.0005):
        input_tensor = Input(shape=x_train_rgbmean.shape[1:])
        
        x = Conv2D(32, (3, 3), use_bias=False, padding="same", kernel_regularizer=l2(reg))(input_tensor)
        x = BatchNormalization(axis=-1)(x)
        
        x = XCEPTION.separable_module(x, 32, reg=reg)
        x = XCEPTION.separable_module(x, 64, reg=reg)
        x = XCEPTION.separable_module(x, 128, reg=reg)
        
        for i in range(8): 
            residual = x
            
            x = Activation('relu')(x)
            x = SeparableConv2D(128, (3, 3), padding='same', use_bias=False, kernel_regularizer=l2(reg))(x)
            x = BatchNormalization(axis=-1)(x)
            x = Activation('relu')(x)
            x = SeparableConv2D(128, (3, 3), padding='same', use_bias=False, kernel_regularizer=l2(reg))(x)
            x = BatchNormalization(axis=-1)(x)
            x = Activation('relu')(x)
            x = SeparableConv2D(128, (3, 3), padding='same', use_bias=False, kernel_regularizer=l2(reg))(x)
            x = BatchNormalization(axis=-1)(x)
            x = add([x, residual])
        
        x = SeparableConv2D(128, (3, 3), padding='same', use_bias=False, kernel_regularizer=l2(reg))(x)
        x = Activation('relu')(x)
        x = BatchNormalization(axis=-1)(x)
        x = SeparableConv2D(256, (3, 3), padding='same', use_bias=False, kernel_regularizer=l2(reg))(x)
        x = Activation('relu')(x)
        x = BatchNormalization(axis=-1)(x)
            
        if include_top:
            x = GlobalAveragePooling2D()(x)
            x = Dropout(0.5)(x)
            x = Dense(num_classes, kernel_regularizer=l2(reg), kernel_constraint=maxnorm(3))(x)
            x = Activation("softmax")(x)
        
        model = Model(input_tensor, x)
        return model

In [8]:
model = XCEPTION.build(include_top=True, reg=0.001)

In [9]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 60, 80, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 60, 80, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 60, 80, 32)   128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
separable_conv2d_1 (SeparableCo (None, 60, 80, 32)   1312        batch_normalization_1[0][0]      
__________________________________________________________________________________________________
activation

In [9]:
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'
from keras.utils import plot_model
plot_model(model, to_file='models/baseline-xception.png', show_shapes=True, show_layer_names=True)

In [10]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [12]:
#filepath=output_path + "progress/inception-weights-{epoch:02d}-{val_acc:.4f}.hdf5"
print(os.getpid())
filepath=output_path + "progress/xception-weights-best.hdf5"
MC = ModelCheckpoint(filepath, monitor='val_acc', verbose=0, save_best_only=True, mode='max')

figPath = os.path.sep.join([output_path, "monitor/{}.png".format(os.getpid())])
jsonPath = os.path.sep.join([output_path, "monitor/{}.json".format(os.getpid())])
TM = TrainingMonitor(figPath, jsonPath=jsonPath, startAt=0)

RLR = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=5, min_lr=0.5e-6)

callbacks = [MC, TM, RLR]

3364


In [13]:
history = model.fit(x_train_rgbmean, y_train_rgbmean,
            batch_size=64,
            epochs=10,
            validation_data=(x_val_rgbmean, y_val_rgbmean),
            shuffle="batch",
            callbacks=callbacks)

Train on 7501 samples, validate on 1607 samples
Epoch 1/10
7501/7501 [==============================] - 38s 5ms/step - loss: 0.2783 - acc: 0.9544 - val_loss: 0.6003 - val_acc: 0.9091
Epoch 2/10
7501/7501 [==============================] - 29s 4ms/step - loss: 0.1545 - acc: 0.9895 - val_loss: 0.1734 - val_acc: 0.9784
Epoch 3/10
7501/7501 [==============================] - 28s 4ms/step - loss: 0.1105 - acc: 0.9953 - val_loss: 0.2877 - val_acc: 0.9543
Epoch 4/10
7501/7501 [==============================] - 29s 4ms/step - loss: 0.0925 - acc: 0.9943 - val_loss: 0.0847 - val_acc: 0.9949
Epoch 5/10
7501/7501 [==============================] - 29s 4ms/step - loss: 0.0632 - acc: 0.9996 - val_loss: 0.0615 - val_acc: 0.9974
Epoch 6/10
7501/7501 [==============================] - 29s 4ms/step - loss: 0.0482 - acc: 0.9999 - val_loss: 0.0444 - val_acc: 0.9991
Epoch 7/10
7501/7501 [==============================] - 28s 4ms/step - loss: 0.0365 - acc: 1.0000 - val_loss: 0.0374 - val_acc: 0.9980
Epoch 8

In [14]:
scores = model.evaluate(x_test_rgbmean, y_test_rgbmean, verbose=0)
print("Train: %.2f%%; Val: %.2f%%; Test: %.2f%%" % 
      (np.max(history.history['acc'])*100, np.max(history.history['val_acc'])*100, scores[1]*100)
     )

Train: 100.00%; Val: 99.91%; Test: 99.43%


In [16]:
model_json = model.to_json()
with open("../output/saved/ucf101_model_xception_99.43.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("../output/saved/ucf101_weight_xception_99.43.hdf5")